# COLAB CODE

In [ ]:
!git clone https://github.com/JavierOramas/thesis_experiments
!mv thesis_experiments/* .

# Load Documents

In [1]:
from pipeline.doc_load.langchain_pdf_loader import load_pdf_data

documents = load_pdf_data("documents/sample_documents")

In [2]:
import re

def split_sentences(text, result_list=[], sep = "."):
    # Split the text into sentences using a simple regex
    # sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    sentences = text.split(sep)
    
    # Initialize variables for tracking the current sentence and the result list
    current_sentence = ''

    # Iterate through the sentences
    for sentence in sentences:
        if sentence == "":
            continue
        # If adding the current sentence exceeds the word limit, start a new sentence
        if len(current_sentence.split()) + len(sentence.split()) > 500:
            result_list.append(current_sentence.strip())
            current_sentence = sentence
        else:
            # Otherwise, continue building the current sentence
            result_list.append(sentence)

    # Add the last sentence to the result list
    if current_sentence:
        result_list.append(current_sentence.strip())

    for s in sentences:
        if len(s.split()) > 500: 
            print(s)
    return result_list




In [3]:
sentences = []
metadata = []
for item in documents:
    for doc in item:
        sentences.append(split_sentences(doc.page_content))
        metadata.append(doc.metadata)

Loading data from PDF document


# Load LLM

In [4]:
from gpt4all import GPT4All
model = GPT4All(model_name="gpt4all-13b-snoozy-q4_0.gguf", model_path="models", allow_download=False)

# Create Topic Model and process sentences

In [5]:
from pipeline.topic_model.k_means_llm import TopicModel
from pipeline.embeddings.basic_embeddings import Embedding
import pandas as pd


tm = TopicModel(llm=model)
embed_model = Embedding()


In [6]:
embeddings = embed_model.encode_many(sentences)
embeddings_df = pd.DataFrame(embeddings)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

# Get Topics

In [7]:
topics = tm.get_topics(sentences, embeddings_df)

/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/pyod/models/ecod.py:25: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return np.nan_to_num(skew_sp(X, axis=axis))
/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5)

Optimal number of topics 8


/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joramas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# for t in topics:
#     print(f"{t}:{topics[t]}" )

# Store Data in Database

In [ ]:
from pipeline.database.chroma_db import ChromaHandler
import uuid

db = ChromaHandler()
collection = db.create_collection(name="test_pipeline")
db.add_embedding_to_database(collection, embeddings=embeddings.tolist(), documents=sentences, metadatas=metadata)

# Detect Relevant information for the topics

In [ ]:
# from langchain.vectorstores import Chroma

# langchain_db = Chroma(client=db.client)
centroids = tm.clusters_centroids.tolist()

top_docs = []
for c in centroids:
    docs = db.query_embeddings(colection, query_embeddings=c, top_k=100)["documents"]
    # docs = langchain_db.max_marginal_relevance_search_by_vector(embedding=c, k=10)
    # print(docs)
    top_docs.append(docs)


In [ ]:
# from ctransformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained(
#     "models/mistral-7b-instruct-v0.1.Q5_K_M.gguf", 
    
#     model_file="models/mistral-7b-instruct-v0.1.Q5_K_M.gguf", 
#     model_type="mistral", 
#     local_files_only=True,
#     gpu_layers=30) 

In [ ]:
words = []
for t in topics:
    for w in topics[t]:
        words = [i[0] for i in w] 
        # print(i[0] for i in w) 
    print(" ".join(words))
    words = []


In [ ]:

# output = model.generate("The capital of France is ", max_tokens=50)

# print(output)

In [ ]:
# # Parameters
# chunk_size = 512  # Adjust based on model's token limit

# # Iterate over top_docs
# for docs in top_docs:
#     # Concatenate documents
#     topic_text = " ".join(docs)

#     # Split into chunks
#     chunks = [topic_text[i:i + chunk_size] for i in range(0, len(topic_text), chunk_size)]

#     # Batch process chunks
#     prompts = [f"Summarize this for a state of the art: {chunk}" for chunk in chunks]
#     try:
#         # Generate summaries
#         outputs = model.generate(prompts)


#     # Concatenate outputs
#     complete_summary = " ".join(outputs)

#     # Post-process if needed
#     complete_summary = post_process(complete_summary)

#     # Use or store the complete summary as needed
#     print(complete_summary)


In [ ]:
for docs in top_docs:
    topic_text = ""
    for doc in docs:
        topic_text += f"{doc}. "

    chunks = [topic_text[i:i + chunk_size] for i in range(0, len(topic_text), chunk_size)]
    for chunk in chunks:
        output = model.generate(f"Sumarize this for a state of the art: {chunk} ")
        print(output)